Template untuk merekapitulasi data untuk suplay data statistik

- menghasilkan rekap jumlah sarana berdasarkan Lokasi, kepemilikan dan jenis (dihitung dari status yang Aktiv/SG )
- rekap dapat di ekpor ke csv/excl file
- cara pemakian tinggal sesuaikan dataset pada line 1 ganti dengan dataset terbaru (dataset tanpa header)
- format dataset menggunakan data di bangwas.web.id (jika berbeda sesuaikan dengan jumlah kolom dan nama kolom)
- data smn belum bisa tercover dikarenakan lokasi tidak bisa diidentifikasi dr daop


In [1]:
import pandas as pd
colnames=['id', 'kode_sarana', 'kode_sarana_lama', 'id_kartu', 'id_jenis', 'id_sub_jenis', 'seri', 'tahun_dinas', 'id_daops', 'id_depo', 'status', 'negara_asal', 'tgl_uji_pertama', 'tgl_masa_uji', 'tgl_masa_perawatan', 'pabrikan', 'id_pemilik', 'created_at', 'last_update' ] 
df_sarana = pd.read_csv('tblsarana_080721.csv', sep=',', names=colnames)
df_sarana.head()


,id,kode_sarana,kode_sarana_lama,id_kartu,id_jenis,id_sub_jenis,seri,tahun_dinas,id_daops,id_depo,status,negara_asal,tgl_uji_pertama,tgl_masa_uji,tgl_masa_perawatan,pabrikan,id_pemilik,created_at,last_update
0,1,SR38401,MTT 2151,0,9,22,85,1984,16,43,Aktiv,NaN,0000-00-00,2020-02-14,0000-00-00,PLASSER AND THEURER,2,0000-00-00 00:00:00,0000-00-00 00:00:00
1,2,SU07501,HKPW,0,9,21,188,1975,18,49,Tidak Aktiv,NaN,0000-00-00,0000-00-00,0000-00-00,(kosong),2,0000-00-00 00:00:00,0000-00-00 00:00:00
2,3,SU39501,EM - 120,0,9,21,190,1995,8,20,Tidak Aktiv,NaN,0000-00-00,2021-08-19,0000-00-00,PLASSER AND THEURER,2,0000-00-00 00:00:00,0000-00-00 00:00:00
3,4,SC38301,HITACHI UDW 90,0,9,20,63,1983,26,66,Aktiv,NaN,0000-00-00,0000-00-00,0000-00-00,HITACHI,2,0000-00-00 00:00:00,0000-00-00 00:00:00
4,5,SC39301,GOTTWALD GS100.05,0,9,20,63,1993,26,48,Aktiv,NaN,0000-00-00,0000-00-00,0000-00-00,GOTTWALD,2,0000-00-00 00:00:00,0000-00-00 00:00:00


In [2]:
col_pemilik=['id', 'nama_pemilik', 'alamat'] 
df_pemilik = pd.read_csv('tblpemiliksaranas.csv', names=col_pemilik, usecols=['id', 'nama_pemilik', 'alamat'])
df_pemilik.head()

,id,nama_pemilik,alamat
0,1,Ditjen Perkeretaapian,Jl. Medan Merdeka Barat No.8 Gambir Jakarta Pusat
1,2,PT Kereta Api Indonesia (Persero),Bandung
2,3,PT Kereta Commuter Indonesia,"Juanda, Jakarta"
3,4,PT Railink,Jakarta
4,5,PT MRT Jakarta,Lebakbulus


In [3]:
df_pemilik.drop(columns =['alamat'], inplace=True)
df_pemilik.head()

,id,nama_pemilik
0,1,Ditjen Perkeretaapian
1,2,PT Kereta Api Indonesia (Persero)
2,3,PT Kereta Commuter Indonesia
3,4,PT Railink
4,5,PT MRT Jakarta


In [4]:
col_daop =['id', 'nama_daop'] 
df_daop = pd.read_csv('tbldaops_080721.csv', names=col_daop, usecols=['id', 'nama_daop'])
df_daop.head()

,id,nama_daop
0,7,DAOP 1 JAKARTA
1,8,DAOP 2 BANDUNG
2,9,DAOP 3 CIREBON
3,10,DAOP 4 SEMARANG
4,11,DAOP 5 PURWOKERTO


# join untuk mengasilkan nama daop dan kepemilikan dari id masing2

In [5]:
join_df = df_sarana.set_index('id_pemilik').join(df_pemilik.set_index('id'), how='left')
join_df2 = join_df.set_index('id_daops').join(df_daop.set_index('id'), how='left')
print(join_df2.head())
print(join_df2.tail())

      id kode_sarana   kode_sarana_lama  id_kartu  id_jenis  id_sub_jenis  \
7  11736     SR31404       MTT CSM 6042         0         9            22   
7     11     SN01501    NR 19 / EX. GGW         0         9            19   
7     12     SN02101           GGR 2429         0         9            19   
7     50     SN06401  NR 8505 / SN01002         0         9            19   
7     53     SN31001           NW 87201         0         9            19   

   seri  tahun_dinas  id_depo       status negara_asal tgl_uji_pertama  \
7     0         2014       71        Aktiv         NaN      2015-04-14   
7   169         1915       57        Aktiv         NaN      0000-00-00   
7   147         1921       14        Aktiv         NaN      0000-00-00   
7   179         2010       15        Aktiv         NaN      0000-00-00   
7   185         2010       17  Tidak Aktiv         NaN      0000-00-00   

  tgl_masa_uji tgl_masa_perawatan             pabrikan           created_at  \
7   2016-04-1

# fungsi untuk mengganti id jenis ke nama jenis sarana

In [6]:
def transform_data(jenis_sarana):
    if jenis_sarana == 1 :
        jenis_sarana = 'Lokomotif'
    elif jenis_sarana == 2 :
        jenis_sarana = 'KRL'
    elif jenis_sarana == 3 :
        jenis_sarana = 'KRD'
    elif jenis_sarana == 4 :
        jenis_sarana = 'Gerbong'
    elif jenis_sarana == 9 :
        jenis_sarana = 'Peralatan Khusus'
    elif jenis_sarana == 11 :
        jenis_sarana = 'Kereta'
    elif jenis_sarana == 19 :
        jenis_sarana = 'RailBus'
    else :
        jenis_sara = 'undefined'
        
    return jenis_sarana 

In [7]:
join_df2['id_jenis'] = join_df2['id_jenis'].apply(lambda x: transform_data(x))
join_df2 = join_df2.rename(columns={"id_jenis": "jenis_sarana"})
print(join_df2.head())
print(join_df2.tail())

      id kode_sarana   kode_sarana_lama  id_kartu      jenis_sarana  \
7  11736     SR31404       MTT CSM 6042         0  Peralatan Khusus   
7     11     SN01501    NR 19 / EX. GGW         0  Peralatan Khusus   
7     12     SN02101           GGR 2429         0  Peralatan Khusus   
7     50     SN06401  NR 8505 / SN01002         0  Peralatan Khusus   
7     53     SN31001           NW 87201         0  Peralatan Khusus   

   id_sub_jenis  seri  tahun_dinas  id_depo       status negara_asal  \
7            22     0         2014       71        Aktiv         NaN   
7            19   169         1915       57        Aktiv         NaN   
7            19   147         1921       14        Aktiv         NaN   
7            19   179         2010       15        Aktiv         NaN   
7            19   185         2010       17  Tidak Aktiv         NaN   

  tgl_uji_pertama tgl_masa_uji tgl_masa_perawatan             pabrikan  \
7      2015-04-14   2016-04-14         0000-00-00  PLASSER AND THE

In [8]:
data_sarana = join_df2[(join_df2["status"] == "Aktiv") & (join_df2["nama_pemilik"] != "Ditjen Perkeretaapian") ]
data_sarana.shape

(11477, 19)

In [9]:
data_sarana = data_sarana[['kode_sarana', 'jenis_sarana', 'nama_pemilik', 'nama_daop']]

In [10]:
data_sarana['nama_daop'].unique()

array(['DAOP 1 JAKARTA', 'DAOP 2 BANDUNG', 'DAOP 3 CIREBON',
       'DAOP 4 SEMARANG', 'DAOP 5 PURWOKERTO', 'DAOP 6 YOGYAKARTA',
       'DAOP 7 MADIUN', 'DAOP 8 SURABAYA', 'DAOP 9 JEMBER',
       'DIVRE I SUMATERA UTARA', 'DIVRE II SUMATERA BARAT',
       'DIVRE III SUMATERA SELATAN', 'PT KAI COMMUTER JABOTABEK',
       'DIVRE IV LAMPUNG', 'PROV DKI JAKARTA'], dtype=object)

# fungsi untuk membuat kategori jawa/sumatera berdasarkan nama_Daop (lihat hasil sebelumnya)

In [11]:
import re
def func(col) :
    if re.match(r'\w+', col['nama_daop'])[0] == "DIVRE" :
        area = "SUMATERA"
    else :
        area = "JAWA"
    return area

In [12]:
data_sarana['lokasi'] = data_sarana.apply(func, axis=1)

In [13]:
data_sarana.lokasi.unique()

array(['JAWA', 'SUMATERA'], dtype=object)

In [14]:
data_sarana[data_sarana['nama_daop'].str.contains('^DIV.*')==True]

,kode_sarana,jenis_sarana,nama_pemilik,nama_daop,lokasi
16,SR38401,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DIVRE I SUMATERA UTARA,SUMATERA
16,SN07701,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DIVRE I SUMATERA UTARA,SUMATERA
16,SN07702,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DIVRE I SUMATERA UTARA,SUMATERA
16,SN07703,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DIVRE I SUMATERA UTARA,SUMATERA
16,SR38409,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DIVRE I SUMATERA UTARA,SUMATERA
...,...,...,...,...,...
26,GT501813,Gerbong,PT TEL,DIVRE IV LAMPUNG,SUMATERA
26,GT501814,Gerbong,PT TEL,DIVRE IV LAMPUNG,SUMATERA
26,GT501815,Gerbong,PT TEL,DIVRE IV LAMPUNG,SUMATERA
26,GT501816,Gerbong,PT TEL,DIVRE IV LAMPUNG,SUMATERA


# fungsi untuk membuat jenis sarana sesuai yg ada di statistik

In [15]:
def subjenis(col) :
    if col['kode_sarana'][2] == "0" and col['jenis_sarana']=='Peralatan Khusus' :
        sub_jenis = "Peralatan Khusus Tanpa Penggerak"
    elif col['kode_sarana'][2] != "0" and col['jenis_sarana']=='Peralatan Khusus' :
        sub_jenis = 'Peralatan Khusus Berpenggerak Sendiri'
    elif col['jenis_sarana'] == "KRL" or col['jenis_sarana']=='KRD' or col['jenis_sarana']=='RailBus' :
        sub_jenis = 'Kereta Berpenggerak Sendiri'
    else :
        sub_jenis = col['jenis_sarana']
    return  sub_jenis


In [16]:
data_sarana['sub_jenis'] = data_sarana.apply(subjenis, axis=1)

In [17]:
data_sarana

,kode_sarana,jenis_sarana,nama_pemilik,nama_daop,lokasi,sub_jenis
7,SN01501,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Tanpa Penggerak
7,SN02101,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Tanpa Penggerak
7,SN06401,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Tanpa Penggerak
7,SI37801,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri
7,SN38203,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri
...,...,...,...,...,...,...
27,K1118138,KRL,PT Angkasa Pura II,PROV DKI JAKARTA,JAWA,Kereta Berpenggerak Sendiri
27,K1118139,KRL,PT Angkasa Pura II,PROV DKI JAKARTA,JAWA,Kereta Berpenggerak Sendiri
27,K1118140,KRL,PT Angkasa Pura II,PROV DKI JAKARTA,JAWA,Kereta Berpenggerak Sendiri
27,K1118141,KRL,PT Angkasa Pura II,PROV DKI JAKARTA,JAWA,Kereta Berpenggerak Sendiri


In [18]:
jumlah_sarana = data_sarana.groupby(['sub_jenis', 'lokasi', 'nama_pemilik'])['kode_sarana'].count().reset_index()
print(jumlah_sarana)

                                sub_jenis    lokasi  \
0                                 Gerbong      JAWA   
1                                 Gerbong  SUMATERA   
2                                 Gerbong  SUMATERA   
3                                  Kereta      JAWA   
4                                  Kereta  SUMATERA   
5             Kereta Berpenggerak Sendiri      JAWA   
6             Kereta Berpenggerak Sendiri      JAWA   
7             Kereta Berpenggerak Sendiri      JAWA   
8             Kereta Berpenggerak Sendiri      JAWA   
9             Kereta Berpenggerak Sendiri      JAWA   
10            Kereta Berpenggerak Sendiri      JAWA   
11            Kereta Berpenggerak Sendiri  SUMATERA   
12            Kereta Berpenggerak Sendiri  SUMATERA   
13            Kereta Berpenggerak Sendiri  SUMATERA   
14                              Lokomotif      JAWA   
15                              Lokomotif  SUMATERA   
16  Peralatan Khusus Berpenggerak Sendiri      JAWA   
17  Perala

In [19]:
rekap = jumlah_sarana.pivot_table(index=['sub_jenis'], columns=['lokasi', 'nama_pemilik'])
rekap

kode_sarana  \
lokasi                                              JAWA   
nama_pemilik                          PT Angkasa Pura II   
sub_jenis                                                  
Gerbong                                              NaN   
Kereta                                               NaN   
Kereta Berpenggerak Sendiri                         12.0   
Lokomotif                                            NaN   
Peralatan Khusus Berpenggerak Sendiri                NaN   
Peralatan Khusus Tanpa Penggerak                     NaN   

                                                                         \
lokasi                                                                    
nama_pemilik                          PT Kereta Api Indonesia (Persero)   
sub_jenis                                                                 
Gerbong                                                          2036.0   
Kereta                                                           1525.0   
Kereta Berpenggerak Sendiri                                       256.0   
Lokomotif                                                         229.0   
Peralatan Khusus Berpenggerak Sendiri                              74.0   
Peralatan Khusus Tanpa Penggerak                                   41.0   

                                                                    \
lokasi                                                               
nama_pemilik                          PT Kereta Commuter Indonesia   
sub_jenis                                                            
Gerbong                                                        NaN   
Kereta                                                         NaN   
Kereta Berpenggerak Sendiri                                 1150.0   
Lokomotif                                                      NaN   
Peralatan Khusus Berpenggerak Sendiri                          NaN   
Peralatan Khusus Tanpa Penggerak                               NaN   

                                                                     \
lokasi                                                                
nama_pemilik                          PT LRT Jakarta PT MRT Jakarta   
sub_jenis                                                             
Gerbong                                          NaN            NaN   
Kereta                                           NaN            NaN   
Kereta Berpenggerak Sendiri                     16.0           96.0   
Lokomotif                                        NaN            NaN   
Peralatan Khusus Berpenggerak Sendiri            1.0            3.0   
Peralatan Khusus Tanpa Penggerak                 NaN            NaN   

                                                  \
lokasi                                             
nama_pemilik                          PT Railink   
sub_jenis                                          
Gerbong                                      NaN   
Kereta                                       NaN   
Kereta Berpenggerak Sendiri                 60.0   
Lokomotif                                    NaN   
Peralatan Khusus Berpenggerak Sendiri        NaN   
Peralatan Khusus Tanpa Penggerak             NaN   

                                                                         \
lokasi                                                         SUMATERA   
nama_pemilik                          PT Kereta Api Indonesia (Persero)   
sub_jenis                                                                 
Gerbong                                                          5339.0   
Kereta                                                            173.0   
Kereta Berpenggerak Sendiri                                        43.0   
Lokomotif                                                         236.0   
Peralatan Khusus Berpenggerak Sendiri                              28.0   
Peralatan Khusus Tanpa Penggerak                                   24.0   

         

In [20]:
data_sarana.columns

Index(['kode_sarana', 'jenis_sarana', 'nama_pemilik', 'nama_daop', 'lokasi',
       'sub_jenis'],
      dtype='object')

In [21]:
rekap.fillna(0, inplace=True)
rekap

kode_sarana  \
lokasi                                              JAWA   
nama_pemilik                          PT Angkasa Pura II   
sub_jenis                                                  
Gerbong                                              0.0   
Kereta                                               0.0   
Kereta Berpenggerak Sendiri                         12.0   
Lokomotif                                            0.0   
Peralatan Khusus Berpenggerak Sendiri                0.0   
Peralatan Khusus Tanpa Penggerak                     0.0   

                                                                         \
lokasi                                                                    
nama_pemilik                          PT Kereta Api Indonesia (Persero)   
sub_jenis                                                                 
Gerbong                                                          2036.0   
Kereta                                                           1525.0   
Kereta Berpenggerak Sendiri                                       256.0   
Lokomotif                                                         229.0   
Peralatan Khusus Berpenggerak Sendiri                              74.0   
Peralatan Khusus Tanpa Penggerak                                   41.0   

                                                                    \
lokasi                                                               
nama_pemilik                          PT Kereta Commuter Indonesia   
sub_jenis                                                            
Gerbong                                                        0.0   
Kereta                                                         0.0   
Kereta Berpenggerak Sendiri                                 1150.0   
Lokomotif                                                      0.0   
Peralatan Khusus Berpenggerak Sendiri                          0.0   
Peralatan Khusus Tanpa Penggerak                               0.0   

                                                                     \
lokasi                                                                
nama_pemilik                          PT LRT Jakarta PT MRT Jakarta   
sub_jenis                                                             
Gerbong                                          0.0            0.0   
Kereta                                           0.0            0.0   
Kereta Berpenggerak Sendiri                     16.0           96.0   
Lokomotif                                        0.0            0.0   
Peralatan Khusus Berpenggerak Sendiri            1.0            3.0   
Peralatan Khusus Tanpa Penggerak                 0.0            0.0   

                                                  \
lokasi                                             
nama_pemilik                          PT Railink   
sub_jenis                                          
Gerbong                                      0.0   
Kereta                                       0.0   
Kereta Berpenggerak Sendiri                 60.0   
Lokomotif                                    0.0   
Peralatan Khusus Berpenggerak Sendiri        0.0   
Peralatan Khusus Tanpa Penggerak             0.0   

                                                                         \
lokasi                                                         SUMATERA   
nama_pemilik                          PT Kereta Api Indonesia (Persero)   
sub_jenis                                                                 
Gerbong                                                          5339.0   
Kereta                                                            173.0   
Kereta Berpenggerak Sendiri                                        43.0   
Lokomotif                                                         236.0   
Peralatan Khusus Berpenggerak Sendiri                              28.0   
Peralatan Khusus Tanpa Penggerak                                   24.0   

         

In [22]:
rekap.columns = rekap.columns.droplevel(0)
rekap = rekap.astype('int64')
rekap

lokasi                                              JAWA  \
nama_pemilik                          PT Angkasa Pura II   
sub_jenis                                                  
Gerbong                                                0   
Kereta                                                 0   
Kereta Berpenggerak Sendiri                           12   
Lokomotif                                              0   
Peralatan Khusus Berpenggerak Sendiri                  0   
Peralatan Khusus Tanpa Penggerak                       0   

lokasi                                                                   \
nama_pemilik                          PT Kereta Api Indonesia (Persero)   
sub_jenis                                                                 
Gerbong                                                            2036   
Kereta                                                             1525   
Kereta Berpenggerak Sendiri                                         256   
Lokomotif                                                           229   
Peralatan Khusus Berpenggerak Sendiri                                74   
Peralatan Khusus Tanpa Penggerak                                     41   

lokasi                                                              \
nama_pemilik                          PT Kereta Commuter Indonesia   
sub_jenis                                                            
Gerbong                                                          0   
Kereta                                                           0   
Kereta Berpenggerak Sendiri                                   1150   
Lokomotif                                                        0   
Peralatan Khusus Berpenggerak Sendiri                            0   
Peralatan Khusus Tanpa Penggerak                                 0   

lokasi                                                               \
nama_pemilik                          PT LRT Jakarta PT MRT Jakarta   
sub_jenis                                                             
Gerbong                                            0              0   
Kereta                                             0              0   
Kereta Berpenggerak Sendiri                       16             96   
Lokomotif                                          0              0   
Peralatan Khusus Berpenggerak Sendiri              1              3   
Peralatan Khusus Tanpa Penggerak                   0              0   

lokasi                                            \
nama_pemilik                          PT Railink   
sub_jenis                                          
Gerbong                                        0   
Kereta                                         0   
Kereta Berpenggerak Sendiri                   60   
Lokomotif                                      0   
Peralatan Khusus Berpenggerak Sendiri          0   
Peralatan Khusus Tanpa Penggerak               0   

lokasi                                                         SUMATERA  \
nama_pemilik                          PT Kereta Api Indonesia (Persero)   
sub_jenis                                                                 
Gerbong                                                            5339   
Kereta                                                              173   
Kereta Berpenggerak Sendiri                                          43   
Lokomotif                                                           236   
Peralatan Khusus Berpenggerak Sendiri                                28   
Peralatan Khusus Tanpa Penggerak                                     24   

lokasi                                                   \
nama_pemilik                          PT Railink PT TEL   
sub_jenis                                                 
Gerbong                                        0    113   
Kereta                                         0      0   
Kereta Berpenggerak Sendiri                   16      0   
Lokomotif    

In [23]:

jumlah_sarana_2 = data_sarana.groupby(['sub_jenis', 'lokasi'])['kode_sarana'].count().reset_index()
rekap_2 = jumlah_sarana_2.pivot_table(index=['sub_jenis'], columns='lokasi')

In [24]:
rekap_2 = rekap_2.astype('int64')


In [25]:
print(rekap_2)

                                      kode_sarana         
lokasi                                       JAWA SUMATERA
sub_jenis                                                 
Gerbong                                      2036     5452
Kereta                                       1525      173
Kereta Berpenggerak Sendiri                  1590       62
Lokomotif                                     229      236
Peralatan Khusus Berpenggerak Sendiri          78       31
Peralatan Khusus Tanpa Penggerak               41       24


In [26]:
rekap_2.index

Index(['Gerbong', 'Kereta', 'Kereta Berpenggerak Sendiri', 'Lokomotif',
       'Peralatan Khusus Berpenggerak Sendiri',
       'Peralatan Khusus Tanpa Penggerak'],
      dtype='object', name='sub_jenis')

In [36]:
#rekap.to_csv('rekapstatistik.csv', sep=';')

aktivkan kode diatas untuk mengekspor ke csv file 

# test code untuk mendefinisakan peralatan khusus yg berpenggerak dan tanpa penggerak

In [28]:
data_perkus = data_sarana[data_sarana['jenis_sarana'] == 'Peralatan Khusus']

In [29]:
data_perkus

,kode_sarana,jenis_sarana,nama_pemilik,nama_daop,lokasi,sub_jenis
7,SN01501,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Tanpa Penggerak
7,SN02101,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Tanpa Penggerak
7,SN06401,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Tanpa Penggerak
7,SI37801,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri
7,SN38203,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri
...,...,...,...,...,...,...
26,SK38204,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DIVRE IV LAMPUNG,SUMATERA,Peralatan Khusus Berpenggerak Sendiri
27,SK31701,Peralatan Khusus,PT MRT Jakarta,PROV DKI JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri
27,SR31803,Peralatan Khusus,PT MRT Jakarta,PROV DKI JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri
27,SR31804,Peralatan Khusus,PT MRT Jakarta,PROV DKI JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri


In [30]:
def sub_jenis(col) :
    if col['kode_sarana'][2] == "0" :
        sub_jenis = "Peralatan Khusus Tanpa Penggerak"
    else :
         sub_jenis = "Peralatan Khusus Berpenggerak Sendiri"
    return  sub_jenis


jenis_sarana2 = ['SN01501', 'SI37801', 'SN38202']
jenis_sarana = 'SN01501'
print (jenis_sarana[2])

0


In [31]:
data_perkus['sub_jenis'] = data_perkus.apply(sub_jenis, axis=1)

<ipython-input-31-f076c7afce70>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_perkus['sub_jenis'] = data_perkus.apply(sub_jenis, axis=1)


In [32]:
data_perkus.head(20)

,kode_sarana,jenis_sarana,nama_pemilik,nama_daop,lokasi,sub_jenis
7,SN01501,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Tanpa Penggerak
7,SN02101,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Tanpa Penggerak
7,SN06401,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Tanpa Penggerak
7,SI37801,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri
7,SN38203,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri
7,SN38202,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri
7,SI38002,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri
7,SI38208,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri
7,SN38201,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri
7,SI38209,Peralatan Khusus,PT Kereta Api Indonesia (Persero),DAOP 1 JAKARTA,JAWA,Peralatan Khusus Berpenggerak Sendiri


In [33]:
rekap_perkus = data_perkus.groupby(['sub_jenis','lokasi'])['kode_sarana'].count().reset_index()

In [34]:
rekap_perkus2 = rekap_perkus.pivot_table(index = ['sub_jenis'], columns='lokasi')

In [35]:
rekap_perkus2

kode_sarana         
lokasi                                       JAWA SUMATERA
sub_jenis                                                 
Peralatan Khusus Berpenggerak Sendiri          78       31
Peralatan Khusus Tanpa Penggerak               41       24